### Classifying Images into Subgroups

In this file we shall attempt to classify images into subgroups. This is intended to help rank the images more effectively against other similar images. This would be added to the front of any ranking pipeline. Because of the labels provided we are only able to train with the AVA dataset (255510 images total).

In [1]:
'''
SCRIPT IMPORTS
'''
#standard ML/Image Processing imports
import numpy as np
import pandas as pd
import math, pandas
import matplotlib.image as mpimg
import matplotlib.pyplot as plt

from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

#pytorch imports
import torch
import torch.optim as optim
import torchvision.models as models

from torch import nn
from torch import optim
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import datasets, transforms

# no one likes irrelevant warnings
import warnings  
warnings.filterwarnings('ignore')

# we load the pretrained model, the argument pretrained=True implies to load the ImageNet weights for the pre-trained model
vgg16 = models.vgg16(pretrained=True)

# root directory where the images are stored
data_dir = "/mnt/md0/reynolds/ava-dataset/"
label_file = "/mnt/md0/reynolds/ava-dataset/AVA_dataset/AVA.txt"
tags_file = "/mnt/md0/reynolds/ava-dataset/AVA_dataset/tags.txt"
limit_lines = 1000000

In [2]:
tag_mapping = {}
f = open(tags_file, "r")
for i, line in enumerate(f):
    if i >= limit_lines:
        break
    line_array = line.split()
    tag_mapping[int(line_array[0])] = line_array[1]
tag_mapping[0] = "Miscellaneous"
print(tag_mapping)

{1: 'Abstract', 24: 'Action', 31: 'Advertisement', 66: 'Analog', 19: 'Animals', 20: 'Architecture', 43: 'Astrophotography', 57: 'Birds', 21: 'Black', 51: 'Blur', 64: 'Camera', 16: 'Candid', 50: 'Children', 2: 'Cityscape', 34: 'Digital', 37: 'Diptych', 49: 'DPChallenge', 12: 'Emotive', 4: 'Family', 3: 'Fashion', 63: 'Fish', 38: 'Floral', 40: 'Food', 53: 'High', 45: 'History', 58: 'Horror', 5: 'Humorous', 46: 'Infrared', 65: 'Insects,', 6: 'Interior', 14: 'Landscape', 62: 'Lensbaby', 22: 'Macro', 56: 'Maternity', 44: 'Military', 59: 'Music', 15: 'Nature', 26: 'Nude', 55: 'Overlays', 33: 'Panoramic', 13: 'Performance', 32: 'Persuasive', 52: 'Photo-Impressionism', 25: 'Photojournalism', 60: 'Pinhole/Zone', 30: 'Political', 17: 'Portraiture', 27: 'Rural', 41: 'Science', 35: 'Seascapes', 47: 'Self', 7: 'Sky', 8: 'Snapshot', 9: 'Sports', 18: 'Still', 61: 'Street', 29: 'Studio', 54: 'Texture', 48: 'Textures', 36: 'Traditional', 39: 'Transportation', 23: 'Travel', 10: 'Urban', 11: 'Vintage', 28

In [3]:
pic_label_dict = {}
f = open(label_file, "r")
for i, line in enumerate(f):
    if i >= limit_lines:
        break
    line_array = line.split()
#     print(line_array)
    picture_name = line_array[1]
    # print(picture_name)
    classifications = (line_array[12:])[:-1]
#     print(classifications)
    for i in range(0, len(classifications)): 
#         classifications[i] = tag_mapping[int(classifications[i])]
        classifications[i] = int(classifications[i])
    # print(max(classifications))
    pic_label_dict[picture_name] = classifications
# print(pic_label_dict)

In [4]:
class ImageFolderWithPaths(datasets.ImageFolder):
    """Custom dataset that includes image file paths. Extends
    torchvision.datasets.ImageFolder
    """

    # override the __getitem__ method. this is the method that dataloader calls
    def __getitem__(self, index):
        # this is what ImageFolder normally returns 
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        # the image file path
        path = self.imgs[index][0]
        # make a new tuple that includes original and the path
        tuple_with_path = (original_tuple + (path,))
#         print(tuple_with_path)
        return tuple_with_path

In [5]:
#Replace the labels in the Dataloaders
class AvaDataset(Dataset):
    def __init__(self, image_path, transform=_transform):
        super(AvaDataset, self).__init__()
        #TODO
    def _find_classes(self, dir):
        #TODO
    def _get_target(self, file_path):
        #TODO
    def make_dataset(self, dir, class_to_idx):
        #TODO
    def get_class_dict(self):
        #TODO
    def __getitem__(self, index):
        #TODO
    def __len__(self):
        #TODO

IndentationError: expected an indented block (<ipython-input-5-208c6633d757>, line 8)

In [6]:
# _transform = transforms.Compose([transforms.ToTensor()])
# Define our data transforms to get all our images the same size
_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

data = ImageFolderWithPaths(data_dir, transform=_transform)

data_loader = torch.utils.data.DataLoader(data)#, num_workers=4)

limit_num_pictures = 1000000

valid_size = 0.2 # percentage of data to use for test set

In [7]:
test_data = data
train_data = data
num_pictures = len(train_data)

# Shuffle pictures and split training set
indices = list(range(num_pictures))

split = int(np.floor(valid_size * num_pictures))
train_idx, test_idx = indices[split:], indices[:split]#rated_indices, bad_indices
#print("Size of training set: {}, size of test set: {}".format(len(train_idx), len(test_idx)))

# Define samplers that sample elements randomly without replacement
train_sampler = SubsetRandomSampler(train_idx)
test_sampler = SubsetRandomSampler(test_idx)

# Define data loaders, which allow batching and shuffling the data
train_loader = torch.utils.data.DataLoader(train_data,
               sampler=train_sampler, batch_size=30)#, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_data,
               sampler=test_sampler, batch_size=30)#, num_workers=4)

# check GPU availability
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# we load the pretrained model, the argument pretrained=True implies to load the ImageNet 
#     weights for the pre-trained model
vgg16 = models.vgg16(pretrained=True)
vgg16.to(device) # loads the model onto the device (CPU or GPU)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [8]:
for param in vgg16.parameters():
    param.requires_grad = False #freeze all convolution weights
network = list(vgg16.classifier.children())[:-1] #remove fully connected layer
network.extend([nn.Linear(4096, 67)]) #add new layer of 4096->100 (rating scale with 1 decimal - similar to 1 hot encoding)
vgg16.classifier = nn.Sequential(*network)

criterion = nn.CrossEntropyLoss() # loss function
optimizer = optim.SGD(vgg16.parameters(), lr=0.4, momentum=0.9) # optimizer

In [ ]:
vgg16.train() # set model to training model
num_epochs = 3 
training_loss = 0
training_accuracy = 0
for epoch in range(num_epochs):
    running_loss = 0.0
    num_correct = 0
    print(train_loader.dataset[0])
    for i, data in enumerate(train_loader,0):
        #print(i)
        for j in range(2): #done so we can utilize both labels
            if limit_num_pictures:
                if i > limit_num_pictures:
                    break
            inputs, _, path = data
            path = path[0]
            path_array = path.split('/')
            pic_name = path_array[-1]
    #         print(pic_name)
    #         print(pic_label_dict[pic_name.split('.')[0]])
    #         label = torch.LongTensor(pic_label_dict[pic_name.split('.')[0]])
            label = pic_label_dict[pic_name.split('.')[0]][j]
#             print(tag_mapping[label])
            label = torch.LongTensor([label])
            print('inputs shape is: {}'.format(inputs.shape))
            print('label shape is: {}'.format(label.shape))
            print('label is : {}'.format(label))
            optimizer.zero_grad()
            output = vgg16(inputs)
            print('output shape is: {} for image #{}'.format(output.shape, i))
    #         print(output, label)
            loss = criterion(output, label)
            running_loss += loss.item()
            _, preds = torch.max(output.data, 1)
            num_correct += (preds == pic_label_dict[pic_name.split('.')[0]][j]).sum().item()
            loss.backward()
            optimizer.step()
    training_loss = running_loss/len(train_loader.dataset)
    training_accuracy = 100 * num_correct/len(train_loader.dataset)

In [ ]:
torch.save(vgg16.state_dict(), 'models/CLASSIFICATION_Feb3_All_AVA_only_training.pt')

In [9]:
vgg16.load_state_dict(torch.load('models/CLASSIFICATION_Feb18_All_AVA_only_training.pt'))

<All keys matched successfully>

In [13]:
limit_num_pictures = 2500
vgg16.eval() # set to prediction mode
testing_loss = 0
testing_accuracy = 0
running_loss = 0.0
num_correct = 0

ratings_data = None
for data in test_loader:
    print(data)
    
    if limit_num_pictures:
        if i > limit_num_pictures:
            break
    inputs, path, _ = data
    path = path[0]
#     label = torch.LongTensor([int(label[0])])

    output = vgg16(inputs)
#     loss = criterion(output, label)

#     running_loss += loss.item()
    _, preds = torch.max(output.data, 1)
#     num_correct += (preds == label).sum().item()
    ratings = output[0].tolist()
    print("\nImage path: {}".format(path))
    print("Classification for test image #{}: {}".format(i, ratings))
    tuple_to_insert = {}
    for n in range(10):
        tuple_to_insert[str(n + 1)] = [ratings[n]]
    tuple_to_insert['file_path'] = [path]
    tuple_to_insert = pandas.DataFrame.from_dict(tuple_to_insert)

    if i == 0:
        ratings_data = tuple_to_insert
    else:
        ratings_data = ratings_data.append(tuple_to_insert, ignore_index=True)
    print(ratings_data.tail(0))
    if i % 200 == 0:
        fig = plt.figure(figsize=(16, 4))
        columns = 3
        rows = 1
        img = mpimg.imread(path)
        fig.add_subplot(rows, columns, 1)
        plt.imshow(img)
        plt.xticks([])
        plt.yticks([])
        plt.show()

# testing_loss = running_loss/len(test_loader.dataset)
# testing_accuracy = 100. * num_correct/len(test_loader.dataset)
ratings_data = ratings_data.set_index('file_path')


[tensor([[[[ 1.6838,  1.6667,  1.6324,  ...,  1.9407,  1.9064,  1.8722],
          [ 1.6495,  1.6495,  1.6495,  ...,  1.9235,  1.8722,  1.8037],
          [ 1.6495,  1.6495,  1.6495,  ...,  1.9235,  1.8722,  1.8550],
          ...,
          [ 1.8208,  1.8208,  1.8208,  ...,  0.3652, -0.2342, -0.6623],
          [ 1.8037,  1.8037,  1.7865,  ...,  0.2624, -0.3541, -0.9877],
          [ 1.7523,  1.7352,  1.7009,  ...,  0.2453, -0.2171, -0.9534]],

         [[ 1.5357,  1.5357,  1.5007,  ...,  1.5182,  1.4832,  1.4307],
          [ 1.5007,  1.5182,  1.5182,  ...,  1.5007,  1.4482,  1.3957],
          [ 1.5182,  1.5182,  1.5182,  ...,  1.4832,  1.4482,  1.4657],
          ...,
          [ 1.7633,  1.7808,  1.7458,  ..., -0.0574, -0.6702, -1.0553],
          [ 1.7458,  1.7283,  1.6933,  ..., -0.1625, -0.7402, -1.3880],
          [ 1.6583,  1.6583,  1.6057,  ..., -0.1975, -0.6176, -1.3704]],

         [[ 1.3154,  1.2457,  1.1934,  ...,  0.9494,  0.8971,  0.8448],
          [ 1.2805,  1.2457, 


Image path: 1
Classification for test image #0: [6808.9931640625, -4866.26953125, -5490.67822265625, -3223.6474609375, -6502.66162109375, 6734.462890625, 2561.2998046875, 1292.581298828125, -1667.1268310546875, -5692.37451171875, -3387.877685546875, -2045.61328125, 5637.71875, -5997.080078125, -1154.7723388671875, -1235.27978515625, 541.7202758789062, -3926.94580078125, 268.7127685546875, 15281.6298828125, 2157.350341796875, -1525.4141845703125, -5233.68701171875, -1709.6614990234375, -8213.2138671875, -2562.50537109375, -2265.681396484375, 7602.5625, -1330.9215087890625, -3348.30810546875, 2198.16845703125, 5064.84326171875, 3243.326416015625, 2315.56201171875, 1275.9124755859375, -4073.347412109375, -3135.10400390625, 3447.738037109375, -9936.185546875, 5898.71630859375, 7184.99560546875, -3628.92333984375, -1617.277587890625, -3416.5791015625, 65.30950927734375, -2788.5830078125, 1728.1512451171875, 4339.82861328125, 4059.336181640625, 1937.939453125, -3179.497802734375, -6016.5937

TypeError: expected str, bytes or os.PathLike object, not NoneType

<Figure size 1152x288 with 0 Axes>